In [1]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [2]:
arg_date = '2021-05-09'
src_format = '%Y-%m-%d'
src_bucket = 'deutsche-boerse-xetra-pds'
trg_bucket = 'paca-bucket-35192'
columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
key = 'xetra_daily_report_' + datetime.today().strftime("%Y%m%d_%H%M%S") + '.parquet'

In [3]:
arg_date_dt = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)

In [4]:
s3 = boto3.resource('s3')
bucket = s3.Bucket(src_bucket)
objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split('/')[0], src_format).date() >= arg_date_dt]

In [5]:
def csv_to_df(filename):
    csv_obj = bucket.Object(key=filename).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=',')
    return df
df_all = pd.concat([csv_to_df(obj.key) for obj in objects], ignore_index=True)

In [6]:
df_all = df_all.loc[:, columns]

In [7]:
df_all.dropna(inplace=True)

## Get opening price per ISIN and day

In [8]:
df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')

## Get closing price per ISIN and day

In [9]:
df_all['closing_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')

## Aggregations

In [10]:
df_all = df_all.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))

## Percent Change Prev Closing

In [11]:
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)

In [12]:
df_all['change_prev_closing_%'] = (df_all['closing_price_eur'] - df_all['prev_closing_price']) / df_all['prev_closing_price'] * 100

In [13]:
df_all.drop(columns=['prev_closing_price'], inplace=True)

In [14]:
df_all = df_all.round(decimals=2)

In [15]:
df_all = df_all[df_all.Date >= arg_date]

## Write to S3

In [16]:
out_buffer = BytesIO()
df_all.to_parquet(out_buffer, index=False)
bucket_target = s3.Bucket(trg_bucket)
bucket_target.put_object(Body=out_buffer.getvalue(), Key=key)

s3.Object(bucket_name='paca-bucket-35192', key='xetra_daily_report_20220220_090311.parquet')

## Reading the uploaded file

In [17]:
for obj in bucket_target.objects.all():
    print(obj.key)

xetra_daily_report_20220220_090311.parquet


In [20]:
prq_obj = bucket_target.Object(key='xetra_daily_report_20220220_090311.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [21]:
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT000000STR1,2021-11-17,37.10,37.10,37.10,37.10,10,NaN
1,AT000000STR1,2021-11-18,37.50,37.50,37.50,37.50,430,1.08
2,AT000000STR1,2021-11-19,36.50,36.65,36.50,36.65,50,-2.27
3,AT000000STR1,2021-11-22,37.60,36.65,36.65,37.60,1570,0.00
4,AT000000STR1,2021-11-23,36.65,36.65,36.65,36.65,0,0.00
...,...,...,...,...,...,...,...,...
620561,XS2376095068,2022-02-18,35.72,35.37,35.00,35.72,2140,-4.19
620562,XS2434891219,2022-02-15,3.87,3.89,3.87,3.92,100,NaN
620563,XS2434891219,2022-02-16,3.89,3.84,3.84,3.89,150,-1.48
620564,XS2434891219,2022-02-17,3.87,3.70,3.70,3.87,7910,-3.41
